In [3]:
import transformer_lens


model = transformer_lens.HookedTransformer.from_pretrained("pythia-14m")

MAX_TOKENS = model.cfg.n_ctx

/home/ubuntu/info-theory-experiments/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loaded pretrained model pythia-14m into HookedTransformer


Idea: we generate two datasets, one of the resid stream of text produced my model and another of resid streams on fixed text.

In [2]:
input_str = "Once upon a time"

input_tokens = model.to_tokens(input_str)[0]

REMANING_TOKENS = MAX_TOKENS - len(input_tokens)

output_str = model.generate(input_str, max_new_tokens=REMANING_TOKENS, stop_at_eos=False)





100%|██████████| 2043/2043 [00:28<00:00, 72.90it/s]


In [3]:
print(model.to_string(model.to_tokens(output_str)[0, -1]))

print(model.to_tokens(output_str).size())

 man
torch.Size([1, 2040])


In [4]:
REMANING_TOKENS = MAX_TOKENS - model.to_tokens(output_str).size()[-1]

print(REMANING_TOKENS)

out2 = model.generate(output_str, max_new_tokens=REMANING_TOKENS, stop_at_eos=False)

8


  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:00<00:00, 68.29it/s]


In [38]:

out2_tokens = model.to_tokens(out2)

_, act_cache = model.run_with_cache(out2_tokens)

print(act_cache["blocks.0.hook_resid_pre"][0].size())

torch.Size([2048, 128])


Saving generated data

In [4]:
import torch
for i in range(6):
    acts = act_cache["blocks.0.hook_resid_pre"][0]
    # save tensor to a file called f"resid_actications_layer_{i}.pt" in activations folder
    torch.save(acts, f"activations/resid_actications_layer_{i}.pt")


                


NameError: name 'act_cache' is not defined

saving non-gen data

In [6]:

for j in range(1, 15):
    with open(f"econ_texts/econ{j}.txt") as f:
        econ_text1 = f.read()

    econ_tokens1 = model.to_tokens(econ_text1)[0][:-1]

    _, cache = model.run_with_cache(econ_tokens1)

    acts = cache["blocks.0.hook_resid_pre"][0]

    torch.save(acts, f"activations2/econ_resid_actications_layer_3_{j}.pt")


In [12]:
# Load one of the saved tensors from the activations folder
loaded_acts = torch.load("activations2/econ_resid_actications_layer_3_8.pt")
print(loaded_acts.size())


torch.Size([739, 128])


/tmp/ipykernel_174882/1508002673.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_acts = torch.load("activations2/econ_resid_actications_layer_3_8.pt")


: 

# now for finding emergent features

In [1]:
import torch
from models import (
    SkipConnectionSupervenientFeatureNetwork
)
from custom_datasets import ResidualStreamDataset
import lovely_tensors as lt

import wandb
import tqdm
from trainers import train_feature_network

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




In [9]:
seed = 42
torch.manual_seed(seed)

resid_config = {
    "torch_seed": seed,
    "dataset_type": "resid",
    "num_atoms": 128,
    "batch_size": 100,
    "train_mode": False,
    "resid_layer_index": 3,
    "use_AI_generated_tokens": True, # if false, use resid streams for 2048 tokens of econ text, otherwise AI generated text
    "train_model_B": False,
    "adjust_Psi": False,
    "clip": 5,
    "feature_size": 1,
    "epochs": 300,
    "start_updating_f_after": 0,
    "update_f_every_N_steps": 1,
    "minimize_neg_terms_until": 0, # if zero I think full psi used, if 9999... then only neg terms
    "downward_critics_config": {
        "hidden_sizes_v_critic": [512, 1024, 1024, 512],
        "hidden_sizes_xi_critic": [512, 1024, 1024, 512],
        "critic_output_size": 32,
        "lr": 1e-2,
        "bias": True,
        "weight_decay": 0,
    },
    
    "decoupled_critic_config": {
        "hidden_sizes_encoder_1": [1024, 1024, 1024],
        "hidden_sizes_encoder_2": [1024, 1024, 1024],
        "critic_output_size": 32,
        "lr": 1e-2,
        "bias": True,
        "weight_decay": 0,
    },
    "feature_network_config": {
        "hidden_sizes": [512, 512, 512, 512, 512],
        "lr": 1e-4,
        "bias": True,
        "weight_decay": 1e-3,
    }
}

dataset = ResidualStreamDataset(
    AI_gen = resid_config["use_AI_generated_tokens"],
    index = resid_config["resid_layer_index"],
)

dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=resid_config["batch_size"],
    shuffle=False
)

feature_network = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=resid_config["num_atoms"],
    feature_size=resid_config["feature_size"],
    hidden_sizes=resid_config["feature_network_config"]["hidden_sizes"],
    include_bias=resid_config["feature_network_config"]["bias"]
).to(device)

project_name = f"resid_layer_{resid_config['resid_layer_index']}_AI_gen_{resid_config['use_AI_generated_tokens']}_emergent"
model_dir_prefix = f"resid_layer_{resid_config['resid_layer_index']}_AI_gen_{resid_config['use_AI_generated_tokens']}_emergent"

skip_model = train_feature_network(
    config=resid_config,
    trainloader=dataloader,
    feature_network_training=feature_network,
    project_name=project_name,
    model_dir_prefix=model_dir_prefix,
)





/home/ubuntu/info-theory-experiments/custom_datasets.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(data_path)


Psi,▁███████████████████████████████████████
decoupled_MI,▁███████████████████████████████████████
downward_MI_0,▁███████████████████████████████████████
downward_MI_1,▁███████████████████████████████████████
downward_MI_10,▁███████████████████████████████████████
downward_MI_100,▁███████████████████████████████████████
downward_MI_101,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
downward_MI_102,▁███████████████████████████████████████
downward_MI_103,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
downward_MI_104,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
downward_MI_105,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
